### Création du Tableau clé valeur

In [31]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time

import psycopg2

from sqlalchemy import create_engine, inspect
from sqlalchemy import DATE, DATETIME, FLOAT, VARCHAR, INTEGER, DateTime

import xml.etree.ElementTree as ET
from inspect import getmembers, isclass, isfunction

from sklearn.preprocessing import LabelEncoder

In [25]:
from fonctions import *

In [28]:
pd.set_option('display.max_row', 200)
pd.set_option('display.max_columns', 50)

folder = 'D:/Fichiers_dev/test_xml/'
folder_export = 'D:/Fichiers_dev/test_xml/tables/'

In [7]:
# xml en tant qu'objet arborescence
# représente la structure hiérarchique complète de l'arbre XML
tree = ET.parse(folder + 'declarations.xml')

# element racine de l'arbre xml, objet Element
# c'est à partir de cet objet qu'on manipule le xml
root = tree.getroot()

In [8]:
print(f"L'arbre contient {len(root)} déclarations.")

L'arbre contient 10893 déclarations.


In [34]:
# creation des listes de stockage
uuids = []
chemins = []
champss = []
valeurs = []
niveaux = []

start_time = time.time()

scan_xml(root)

dictionnaire = {
    'id_declaration' : uuids,
    'chemins' : chemins,
    'champs' : champss,
    'valeurs' : valeurs,
    'niveaux' : niveaux
}

data = pd.DataFrame(dictionnaire)

end_time = time.time()

temps_execution1 = end_time - start_time

print(f"Le temps d'execution est de {temps_execution1}")

print(f"{data.shape[0]} lignes, {data.shape[1]} colonnes.")

Le temps d'execution est de 6.245836973190308
3010356 lignes, 5 colonnes.


In [35]:
# nettoyage des valeurs tabulation "\t"
data = data.drop(data[data['valeurs'].str.contains('\t')].index)
data.head()

,id_declaration,chemins,champs,valeurs,niveaux
0,4344aaa1-874d-4e6d-9b1a-45f7725b710c,declaration/dateDepot,dateDepot,11/07/2022 15:40:13,1
1,4344aaa1-874d-4e6d-9b1a-45f7725b710c,declaration/uuid,uuid,4344aaa1-874d-4e6d-9b1a-45f7725b710c,1
2,4344aaa1-874d-4e6d-9b1a-45f7725b710c,declaration/origine,origine,ADEL,1
3,4344aaa1-874d-4e6d-9b1a-45f7725b710c,declaration/complete,complete,true,1
6,4344aaa1-874d-4e6d-9b1a-45f7725b710c,declaration/attachedFiles/attachedFiles/fileName,fileName,VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML,3


In [36]:
data.shape

(1944263, 5)

In [37]:
# pour toutes les valeurs qui contiennent '\n', efface les '\n' puis supprime les espaces au début et à la fin de la chaine
data.loc[data['valeurs'].str.contains('\n'), 'valeurs'] = data.loc[data['valeurs'].str.contains('\n'),
                                                                   'valeurs'].str.replace('\n', '').str.strip()

In [38]:
# suppression des champs en base 64, chemins ou il y a 'b64'
data = data.drop(data[data['chemins'].str.contains('b64')].index)
data = data.drop(data[data['chemins'].str.contains('base64')].index)

In [39]:
data.shape

(1944208, 5)

In [40]:
# modification chemin
data['chemins'] = data['chemins'].str.replace('declaration/', '')

In [45]:
# suppression niveau et reset index
data = data.drop(columns='niveaux').reset_index(drop=True)

In [46]:
# export
data.to_csv(folder_export + 'data_nettoyees.csv', index=False)